# 📌 Set up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 📌 Extract: O E do ETL

## 1️⃣ Lendo em excel

In [ ]:
df = pd.read_excel("planilha.excel", sheet_name="aba")

##  2️⃣ Lendo em csv

In [ ]:
df = pd.read_csv("arquivo.csv")

## 3️⃣ Lendo de Google Sheets

In [ ]:
import gspread #Importa a biblioteca gspread, usada para interagir com o Google Sheets.
from google.colab import auth #importa uma forma de autenticação para o google colab
auth.authenticate_user() # Autenticar no Google (irá solicitar permissão)
from google.auth import default #Permite obter as credenciais padrão do ambiente (útil se estiver rodando no Google Colab ou em um servidor com autenticação do Google)
creds, _ = default() #Atribui as credenciais de autenticação automaticamente.

def aquisicao_google_sheets(sheet_id, sheet_name):
    SHEET_ID = sheet_id
    SHEET_NAME = sheet_name
    gc = gspread.authorize(creds)
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    rows = worksheet.get_all_records()
    df = pd.DataFrame(rows)
    return df

df = aquisicao_google_sheets("SHEET_ID", "SHEET_NAME")

# 📌 Avaliação Sumária

## 1️⃣ Verificando as Dimensões do DataFrame

In [ ]:
# retorna o número de linhas e colunas do DataFrame.
df.shape

In [ ]:
# retornar primeiros valores
df.head(10)

In [ ]:
# retornar últimos valores
df.tail(10)

## 2️⃣ Identificando os Tipos de Dados das Colunas

In [ ]:
# Mostra informações gerais sobre o DataFrame.
df.info()

In [ ]:
# ver quantas linhas nula
df.isna().sum()

In [ ]:
# quiser saber apenas dos tipos
df.dtypes

## 3️⃣  Obtendo Estatísticas Descritivas das Colunas Numéricas

In [ ]:
# estatísticas resumidas
df.describe()

In [ ]:
# filtra colunas numéricas, para ter estatísticas aplicáveis a dados numéricos
df.describe(include=[float, int])

## 4️⃣  Obtendo Estatísticas Descritivas das Colunas Categóricas

In [ ]:
# quais os valores existentes na coluna
df['coluna'].unique()

In [ ]:
# exibe estatísticas resumidas para todas as colunas categóricas
df.describe(include=[object])

In [ ]:
# usando o método .value_counts() em uma coluna específica é possível verificar sua distribuição de valores
df["coluna"].value_counts()

In [ ]:
# caso queira contar números de nulos, adicione como parâmetro do value_counts()
df["coluna"].value_counts(dropna=False)

In [ ]:
# caso queira na verdade o percentual, adicione como parâmetro de value_counts()
df["coluna"].value_counts(dropna=False, normalize=True)

In [ ]:
# se quer multiplicar por 100, basta multiplicar por 100
df["coluna"].value_counts(dropna=False, normalize=True) * 100

# 📌 Transform: Primeiro Tratar

## 1️⃣ Ao nível de colunas

### 💡 Acessar nomes de coluna

In [ ]:
# visualizar os nomes das colunas do DataFrame
df.columns

### 💡 Mudar nomes de coluna

In [ ]:
# permite passar um dicionário que atribue a cada coluna existente um novo nome
df = df.rename(columns={
    'nome_antigo_1': 'nome_novo_1',
    'nome_antigo_2': 'nome_novo_2',
    })

In [ ]:
# Se os nomes das colunas estiverem inconsistentes, podemos formatá-los automaticamente para minúsculo
# df.columns = df.columns.str.lower()

In [ ]:
# se quisermos remover espaços e deixar padronizado:
# df_receitas.columns = df_receitas.columns.str.strip().str.replace(" ", "_")

### 💡 Colocar a tipologia da coluna

In [ ]:
# Se uma coluna que deveria ser string foi carregada como número, podemos convertê-la usando a função astype
df['coluna'] = df['coluna'].astype(str)

In [ ]:
# converter a coluna 'data_lancamento' para o tipo datetime
df['coluna'] = pd.to_datetime(df['coluna'], format='%Y/%m/%d', errors='coerce')

[Clique para ver a documentação](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

### 💡 Mudar ordem das colunas

In [ ]:
# definir uma nova ordem das colunas especificando a sequência

df = df[[
    "coluna_1", "coluna_2"
]]

In [ ]:
# Se quisermos organizar automaticamente as colunas em ordem alfabética
# df = df[sorted(df.columns)]

### 💡 Descartando uma coluna desnecessária

In [ ]:
# retirar uma coluna específica
df = df.drop(columns=["coluna"])

In [ ]:
# retirar mais de uma coluna específica
df = df.drop(columns=["coluna_1", "coluna_2"])

### 💡 Descartando uma coluna com nulidade

In [ ]:
# axis = 1 identifica para dropar colunas, how identifica o critério de nulidade
df = df.dropna(axis=1, how="all")

[Clique para ver documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)

## 2️⃣ Ao nível de linhas

### 💡 Substituir linhas

In [ ]:
# substituição condicional simples
df["coluna"] = np.where(df["coluna"] == 'X', "Valor", df["coluna"])

In [ ]:
# substituição condicional complexa

# condicoes
condicoes = [df["coluna"] == "X",
             df["coluna"] == "Y"
             ]

# Definir os valores correspondentes para cada condição. Tem que ter um valor para cada condição
valores = ["Valor_1", "Valor_2"]

# Aplicar a substituição com np.select() com um valor default
df["coluna"] = np.select(condicoes, valores, default= df["coluna"])

In [ ]:
# Substituir diretamente valores específicos para texto
df["coluna"] = df["coluna"].replace({"X": "Valor_1", "Y": "Valor_2"})

In [ ]:
# Criar um dicionário de mapeamento e aplicar com map().
mapa = {"X": "Valor_1", "Y": "Valor_2"}
df["coluna"] = df["coluna"].map(mapa)

In [ ]:
# Função personalizada para padronizar a coluna

def padronizar(valor):
    if valor in ["X"]:
        return "Valor_1"
    elif valor in ["Y"]:
        return "Valor_2"
    return valor #retorna o valor original caso não seja alterado

# Aplicando a função com apply()
df["coluna"] = df["coluna"].apply(padronizar)

### 💡 Imputar linhas

In [ ]:
# Substituir valores nulos por "Não informado"
df["coluna"] = np.where(df["coluna"].isna(), "Não informado", df["coluna"])

In [ ]:
# Preenchendo nulos com a moda (valor mais frequente) da coluna
df["coluna"] = df["coluna"].fillna("Não informado")

[Clique para ver a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)

### 💡 Filtrar linhas

In [ ]:
# filtragem simples de coluna com valor X
df.loc[df["coluna"] == "X"]

In [ ]:
# filtragem simples posicional, retorna as primeiras 5 linhas, Porque o index máximo é 4? Porque em Python a indexação começa no 0.
df.iloc[0:5]

In [ ]:
# filtragem simples ao perguntar pelas linhas com a função .query()
df.query("coluna == 'X'")

In [ ]:
# filtragem composta de coluna
df.loc[(df["coluna"] == "X") & (df["coluna"] == "Y")]
df.loc[(df["coluna"] == "X") | (df["coluna"] == "Y")]

In [ ]:
# filtragem composta de coluna com query
df.query("coluna == 'X' & coluna == 'Y'")
df.query("coluna == 'X' | coluna == 'Y'")

In [ ]:
# filtrar composta de texto que está dentro de uma lista
df[df["coluna"].isin(["X", "Y"])]

In [ ]:
# filtrar composta de texto que contém
df[df['coluna'].str.contains('X', case=False)]

In [ ]:
# filtrar valor que está ausente (NaN)
df[df["coluna"].isna()]

In [ ]:
# filtrar valor que está NÃO ausente (NaN)
df[df["coluna"].notna()]

In [ ]:
# filtrar por data
df[df["coluna"].between("data_1", "data_2")]

### 💡 Deduplicar linhas

In [ ]:
# Verifica se há linhas duplicadas (considerando todas as colunas)
df.duplicated().sum()

In [ ]:
# exibir apenas as linhas duplicadas
df[df.duplicated()]

In [ ]:
# remover duplicatas e manter apenas a primeira ocorrência
df = df.drop_duplicates()

In [ ]:
# Remover duplicatas considerando apenas uma coluna
df = df.drop_duplicates(subset=["coluna"])

In [ ]:
# Mantendo apenas a última ocorrência
df = df.drop_duplicates(subset=["coluna"], keep="last")

[Clique para acessar a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

### 💡 Ordenar linhas

In [ ]:
# ordenar pelo valor de venda
df = df.sort_values(by="coluna_data")

In [ ]:
# ordenar do maior para o menor valor de venda
df = df.sort_values(by="venda_valor", ascending=False)

In [ ]:
# ordenar primeiro pela coluna_1 e depois pela coluna_2
df = df.sort_values(by=["coluna_1", "coluna_2"])

In [ ]:
# ordenar primeiro pela coluna_1 do maior para menor e depois pela coluna_2 do menor para maior
df = df.sort_values(by=["coluna_1", "coluna_2"], ascending = [False, True])

In [ ]:
# ordenar primeiro pela coluna_1 do maior para menor e depois pela coluna_2 do menor para maior, reindexando
df = df.sort_values(by=["coluna_1", "coluna_2"], ascending = [False, True]).reset_index(drop=True)

# 📌 Transform: Depois Enriquecer

## 1️⃣ Ao nível de colunas

### 💡 Coluna Condicional ou Personalizada

In [ ]:
# Criando a coluna de idade baseada na data de nascimento. Pegamos o ano atual e subtraímos o ano de nascimento, obtendo a idade.
from datetime import datetime
df["idade"] = datetime.today().year - pd.to_datetime(df["data_nascimento"]).dt.year

In [ ]:
# Criando categorias de idade com pd.cut()
df["faixa_etaria"] = pd.cut(
    df["idade"],
    bins=[0, 17, 35, 60, 100],  # Limites das faixas etárias
    labels=["0-17", "18-35", "36-60", "61-100"]
)

In [ ]:
# Criando condições para as faixas etárias
condicoes = [
    df["idade"] < 18,
    (df["idade"] >= 18) & (df["idade"] <= 35),
    (df["idade"] > 35) & (df["idade"] <= 60),
    df["idade"] > 60
]

# Definindo os rótulos correspondentes
categorias = ["0-17", "18-35", "36-60", "61-100"]

# Criando a nova coluna com np.select()
df["faixa_etaria"] = np.select(condicoes, categorias, default="Desconhecido")

### 💡 Aplicando funções

In [ ]:
# Função para categorizar a idade
def classificar_faixa_etaria(idade):
    if idade < 18:
        return "0-17"
    elif idade <= 35:
        return "18-35"
    elif idade <= 60:
        return "36-60"
    elif idade > 60:
        return "61-100"
    else:
        return "Desconhecido"

In [ ]:
# Aplicando a função com apply()
df["faixa_etaria"] = df["idade"].apply(classificar_faixa_etaria)

## 2️⃣ Ao nível da tabela

### 💡 Junção de tabelas (Join / Merge)

##### 🚩 **INNER JOIN**

In [ ]:
# valor comuns aos dois dataframe com chaves com mesmo nome
pd.merge(df_esquerda, df_direita, on='chave', how='inner')

##### 🚩 **LEFT OUTER JOIN**

In [ ]:
# valor comuns aos dois daframe e valor do dataframe da esquerda
pd.merge(df_esquerda, df_direita, on='chave', how='left')

In [ ]:
# valor comuns aos dois dataframe com chaves com diferentes nomes e mostrando como ficaram os merges
pd.merge(df_esquerda, df_direita, left_on="chave_esquerda", right_on="chave_direita", how="left", indicator=True)

##### 🚩 **RIGHT OUTER JOIN**

In [ ]:
# valor comuns aos dois daframe e valor do dataframe da direita
pd.merge(df_esquerda, df_direita, on='chave', how='right')

##### **🚩 FULL OUTER JOIN**

In [ ]:
# valor comuns aos dois daframe e valor do dataframe da direita e da esqueda
pd.merge(df_esquerda, df_direita, on='chave', how='outer')

##### **🚩 COMBINING**

In [ ]:
# empilhar os dois dataframes
pd.concat([df_esquerda, df_direita])

[Clique para consultar a documentação](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)

### 💡 Agregações

In [ ]:
# agregar por uma coluna
df_agg = df.groupby('coluna_agregante')['medida'].mean()

In [ ]:
# agregar por mais de uma coluna
df_agg = df.groupby(['coluna_agregante_1', 'coluna_agregante_2'])['medida'].mean()

In [ ]:
# agregar por mais de uma medida
df_agg = df.groupby("coluna_agregante")["medida"].agg(["mean", "min"])

In [ ]:
# agregar por uma função

def classificar_grupo(grupo):
    media = grupo["medida"].mean()
    if media > 8:
        return "Excelente"
    elif media >= 6:
        return "Regular"
    else:
        return "Ruim"

# Aplicando por grupo de Sexo
df_agg = df.groupby("coluna_agregante").apply(classificar_grupo)

### 💡 Multi-índices

In [ ]:
multi_indices = df.set_index(['indice_1', 'indice_2'])
multi_indices

### 💡 Stack & Unstack

#### `Stack`

In [ ]:
empilhado = df.stack(0)
empilhado

#### `Unstack`

In [ ]:
desempilhado = empilhado.unstack(0)
desempilhado

### 💡 Tabelas Dinâmicas

- index: Coluna que vai para o eixo das linhas (índice da tabela)
- columns: Coluna que será transformada em colunas
- values: Coluna cujos valores serão agregados
- aggfunc: Função de agregação (ex: 'mean', 'sum', 'count', etc.)

In [ ]:
df_pivot = df.pivot_table(
    index="coluna_agregante",
    values='coluna_medida'
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values='medida'
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values='coluna_medida',
    aggfunc="medida"
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values='coluna_medida',
    aggfunc=["medida_1", "medida_2"]
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values=['coluna_medida_1', 'coluna_medida_2'],
    aggfunc={'coluna_medida_1': 'medida_1','coluna_medida_2': ['medida_1','medida_2']}
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values=['coluna_medida_1', 'coluna_medida_2'],
    columns=['coluna_coluna_1', 'coluna_coluna_2'], # colunas para unstack
    aggfunc={'coluna_medida_1': 'medida_1','coluna_medida_2': ['medida_1','medida_2']}
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values=['coluna_medida_1', 'coluna_medida_2'],
    columns=['coluna_coluna_1', 'coluna_coluna_2'], # colunas para unstack
    aggfunc={'coluna_medida_1': 'medida_1','coluna_medida_2': ['medida_1','medida_2']},
    fill_value=0 # preencher valores nulos com 0
)

df_pivot

In [ ]:
df_pivot = df.pivot_table(
    index= ["coluna_agregante_1", "coluna_agregante_2"],
    values=['coluna_medida_1', 'coluna_medida_2'],
    columns=['coluna_coluna_1', 'coluna_coluna_2'], # colunas para unstack
    aggfunc={'coluna_medida_1': 'medida_1','coluna_medida_2': ['medida_1','medida_2']},
    fill_value=0,
    margins=True, #adicionar totais
    margins_name='Total' #modificar nome do total
)

df_pivot

[Clique para consultar a documentação](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)

# 📌 Um recurso rápido

In [ ]:
pip install https://github.com/ydataai/pandas-profiling/archive/master.zip --quiet

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [ ]:
profile

In [ ]:
profile.to_file("meu_relatorio.html")